In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable

#############Strategy Hyperparameters##################
MA_LENGTH = 80
ST_DEV = 2
SQUEEZE_LOOK_BACK = 250
SQUEEZE_MEMORY = 50
ATR_LENGTH = 15
ATR_STOP = 6
RISK_FREE_RATE = 0.05  

In [16]:
Results = pd.DataFrame(columns=['CAGR','Sharpe','MaxDD','AllTrdes','WinbyLoss','PercProfitable'])
params = pd.DataFrame(columns=['MA_LENGTH','ST_DEV','SQUEEZE_LOOK_BACK','SQUEEZE_MEMORY','ATR_LENGTH','ATR_STOP'])
final = pd.DataFrame()
P = pd.DataFrame()

for MA_LENGTH in range(70,85,5):
    for ST_DEV in range(20,25,5):
        for SQUEEZE_LOOK_BACK in range(25,150,25):
            for SQUEEZE_MEMORY in range(50,150,50):
                for ATR_LENGTH in range(10,11):
                    for ATR_STOP in range(5,6):
                        params.loc[len(params)] = [MA_LENGTH,ST_DEV/10,SQUEEZE_LOOK_BACK,SQUEEZE_MEMORY,ATR_LENGTH,ATR_STOP]
                        P,Results.loc[len(Results)] = Parameter_Tune1(MA_LENGTH,ST_DEV/10,SQUEEZE_LOOK_BACK,SQUEEZE_MEMORY,ATR_LENGTH,ATR_STOP)
                        final = pd.concat([params, Results], axis=1)
                        final.to_csv("documents/Parameters_test.csv")

In [15]:

def Parameter_Tune1(MA_LENGTH,ST_DEV,SQUEEZE_LOOK_BACK,SQUEEZE_MEMORY,ATR_LENGTH,ATR_STOP):

    total_trades=0
    profits = 0
    losses = 0
    profitbars = 0
    lossbars = 0
    port_total_trades = []
    port_profits = []
    port_losses = []
    port_profit_bars = []
    port_loss_bars = []

    Portfolio = pd.DataFrame()
    for filename in os.listdir('../iRage_Round_2_Assignment/data/test/'):
        if filename.endswith(".csv"): 
            df = pd.read_csv('../iRage_Round_2_Assignment/data/test/' + filename)
            df['Date'] = pd.to_datetime(df['Date'])

            preprocess_with_hyperparameters(df,MA_LENGTH,ST_DEV,SQUEEZE_LOOK_BACK,SQUEEZE_MEMORY,ATR_LENGTH)

            total_trades,profits,losses,profitbars,lossbars = backtest(df,ATR_STOP)

            df.set_index(['Date'],inplace=True)

            Portfolio[filename] = df['PortRets']

            port_total_trades.append(total_trades)
            port_profits.append(profits)
            port_losses.append(losses)
            port_profit_bars.append(profitbars)
            port_loss_bars.append(lossbars)

    Portfolio.fillna(0,inplace=True)
    Portfolio = Portfolio
    Portfolio.index = pd.to_datetime(Portfolio.index)

    port_profit_bars = [item for sublist in port_profit_bars for item in sublist]
    port_loss_bars = [item for sublist in port_loss_bars for item in sublist]
    port_profits  = [item for sublist in port_profits for item in sublist]
    port_losses  = [item for sublist in port_losses for item in sublist]



    testing_period_years = (Portfolio.index[-1] - Portfolio.index[0]).days/360
    CAGR = ((Portfolio.sum(axis=1).iloc[-1]/Portfolio.sum(axis=1).iloc[0])**(1/testing_period_years)) - 1 

    perc_profitable_trades = len(port_profits)/sum(port_total_trades)
    win_by_loss_amt_ratio = sum(port_profits)/sum(port_losses)*-1
    Total_number_of_trades = sum(port_total_trades)
    sharpe_ratio = (CAGR - RISK_FREE_RATE)/(Portfolio.sum(axis=1).std()/(100*len(Portfolio.columns)))

    drawdown = ((Portfolio.sum(axis=1) - Portfolio.sum(axis=1).cummax())/Portfolio.sum(axis=1).cummax())
    drawdown[drawdown == -np.inf] = 0
    max_drawdown = (drawdown.min())

    return Portfolio, [CAGR,sharpe_ratio,max_drawdown,Total_number_of_trades,win_by_loss_amt_ratio,perc_profitable_trades]

In [13]:
def Parameter_Tune(MA_LENGTH,ST_DEV,SQUEEZE_LOOK_BACK,SQUEEZE_MEMORY,ATR_LENGTH,ATR_STOP):
    total_trades=0
    profits = 0
    losses = 0
    profitbars = 0
    lossbars = 0
    port_total_trades = []
    port_profits = []
    port_losses = []
    port_profit_bars = []
    port_loss_bars = []

    Portfolio = pd.DataFrame()
    for filename in os.listdir('/Users/vipultanwar/Documents/iRage_Round_2_Assignment'):
        if filename.endswith(".csv"): 
            df = pd.read_csv(filename)
            preprocess_with_hyperparameters(df,MA_LENGTH,ST_DEV,SQUEEZE_LOOK_BACK,SQUEEZE_MEMORY,ATR_LENGTH)

            total_trades,profits,losses,profitbars,lossbars = backtest(df,ATR_STOP)

            df.set_index(['Date'],inplace=True)

            Portfolio[filename] = df['PortRets']
            port_total_trades.append(total_trades)
            port_profits.append(profits)
            port_losses.append(losses)
            port_profit_bars.append(profitbars)
            port_loss_bars.append(lossbars)

    Portfolio.fillna(0,inplace=True)
    Portfolio = Portfolio
    Portfolio.index = pd.to_datetime(Portfolio.index)

    port_profit_bars = [item for sublist in port_profit_bars for item in sublist]
    port_loss_bars = [item for sublist in port_loss_bars for item in sublist]
    port_profits  = [item for sublist in port_profits for item in sublist]
    port_losses  = [item for sublist in port_losses for item in sublist]

    testing_period_years = (Portfolio.index[-1] - Portfolio.index[0]).days/360

    CAGR = ((Portfolio.sum(axis=1).iloc[-1]/Portfolio.sum(axis=1).iloc[0])**(1/testing_period_years)) - 1 

    perc_profitable_trades = len(port_profits)/sum(port_total_trades)
    win_by_loss_amt_ratio = sum(port_profits)/sum(port_losses)*-1
    Total_number_of_trades = sum(port_total_trades)
    sharpe_ratio = (CAGR - RISK_FREE_RATE)/(Portfolio.sum(axis=1).std())

    drawdown = ((Portfolio.sum(axis=1) - Portfolio.sum(axis=1).cummax())/Portfolio.sum(axis=1).cummax())
    drawdown[drawdown == -np.inf] = 0
    max_drawdown = (drawdown.min())

    return Portfolio, [CAGR,sharpe_ratio,max_drawdown,Total_number_of_trades,win_by_loss_amt_ratio,perc_profitable_trades]

In [14]:

Portfolio, _ = Parameter_Tune(20,2,50,100,14,3)
testing_period_years = (Portfolio.index[-1] - Portfolio.index[0]).days/360

CAGR = ((Portfolio.sum(axis=1).iloc[-1]/Portfolio.sum(axis=1).iloc[0])**(1/testing_period_years)) - 1 
    
perc_profitable_trades = len(port_profits)/sum(port_total_trades)
win_by_loss_amt_ratio = sum(port_profits)/sum(port_losses)*-1
Total_number_of_trades = sum(port_total_trades)
sharpe_ratio = (CAGR - RISK_FREE_RATE)/(Portfolio.sum(axis=1).std())

drawdown = ((Portfolio.sum(axis=1) - Portfolio.sum(axis=1).cummax())/Portfolio.sum(axis=1).cummax())
drawdown[drawdown == -np.inf] = 0
max_drawdown = (drawdown.min())

x = PrettyTable()

x.field_names = ["Metrics", "Value", "Unit"]

x.add_row(["Strategy CAGR",round(CAGR*100,2),"% per annum"])
x.add_row(["Maximum Drawdown",round(max_drawdown*100,2),"%"])
x.add_row(["Percent Profitable Trades",round(perc_profitable_trades*100,2),"%"])
x.add_row(["Avg. Win / Avg. Loss amount Ratio",round(win_by_loss_amt_ratio,2),"$"])
x.add_row(["Sharpe Ratio",round(sharpe_ratio,2),""])
x.add_row(["Total Number of Trades",Total_number_of_trades,"trades"])
x.add_row(["Winning Trades",len(port_profits),"trades"])
x.add_row(["Losing Trades",len(port_losses),"trades"])
x.add_row(["Even Trades",Total_number_of_trades -len(port_profits) - len(port_losses),"trades"])
x.add_row(["Largest Winning Trade",round(max(port_profits),2),"for $100 initial invesetment"])
x.add_row(["Largest Losing Trade",round(min(port_losses),2),"for $100 initial invesetment"])
x.add_row(["Avg Bars in Winning Trades",round(sum(port_profit_bars)/len(port_profit_bars),2),"bars"])
x.add_row(["Avg Bars in Losing Trades",round(sum(port_loss_bars)/len(port_loss_bars),2),"bars"])

fig = plt.figure(figsize=(20,8))
ax = fig.gca()

plt.grid()
plt.xlabel("Year")
plt.ylabel("Equity")
plt.plot(Portfolio.sum(axis=1)/(len(Portfolio.columns)))

print(x)


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:

def preprocess_with_hyperparameters(df,MA_Length,St_Dev,Squeeze_Look_Back,Squeeze_Memory,ATR_Length):
    get_bollinger_bands(df,MA_Length,St_Dev)
    get_BBand_Cross_Signal(df)
    get_volatility_sqeeze_signal(df,Squeeze_Look_Back,Squeeze_Memory)
    get_n_days_ATR(df,ATR_Length)
    df['returns'] = df['Close'].pct_change()
    TrendExitconditions = [
            (df['Close'].shift(1) <  df['MA'].shift(1)),
            (df['Close'].shift(1) >  df['MA'].shift(1)),
            ]
    TrendExitvalues = [-1,1]

    df['TrendExit'] = 0
    df['TrendExit'] = np.select(TrendExitconditions, TrendExitvalues)
    
    return df
    

In [ ]:
def backtest(df,ATR_Stop_):
    
    df.reset_index(inplace=True)
    Position = 0
    stop_loss = 0 
    rets = []
    ATR_Stop = ATR_Stop_   #More negative more aggressive
    value_before_trade = 0
    count_trades = 0
    profits_bag = []
    losses_bag = []
    profit_bars = []
    loss_bars = []
    Capital = 100
    trade_start_bar = 0
    for n, row in df.iterrows():

#         if(Position != 0):
#             print(Position,"|",round(row.Close,2),"|",round(stop_loss,2),"|",round(rets[-1],2),"|",row.TrendExit)

        if(Position > 0):
            trade_start_bar += 1
            Capital = (1+row.returns)*Capital
            rets.append(Capital)
            if((row.TrendExit < 0) | ((row.Close < stop_loss) & (stop_loss > 0))):
                Position = 0
                stop_loss = 0
                count_trades += 1
                if(Capital > value_before_trade):
                    profits_bag.append(Capital - value_before_trade)
                    profit_bars.append(trade_start_bar)
                    trade_start_bar = 0
                else:
                    losses_bag.append(Capital - value_before_trade)
                    loss_bars.append(trade_start_bar)
                    trade_start_bar = 0        
                    
                
        elif(Position < 0):
            trade_start_bar +=1
            Capital = (1-row.returns)*Capital
            rets.append(Capital)
            if((row.TrendExit > 0) | ((row.Close > stop_loss)& (stop_loss > 0))):
                Position = 0
                stop_loss = 0
                count_trades += 1
                if(Capital > value_before_trade):
                    profits_bag.append(Capital - value_before_trade)
                    profit_bars.append(trade_start_bar)
                    trade_start_bar = 0  
                else:
                    losses_bag.append(Capital - value_before_trade)
                    loss_bars.append(trade_start_bar)
                    trade_start_bar = 0            
                  
        else:
            rets.append(Capital)

            if((row.IsVolatilitySqueeze)*(row.BandCrossSignal) > 0):
                Position = 1
                trade_start_bar = 1
                value_before_trade = Capital
                stop_loss = row.Close - (row.ATR*(ATR_Stop))
                
            elif((row.IsVolatilitySqueeze)*(row.BandCrossSignal) < 0):
                Position = -1
                trade_start_bar = 1
                stop_loss = row.Close + (row.ATR*(ATR_Stop))
                value_before_trade = Capital


    df['PortRets'] = 0
    df['PortRets'] = pd.Series(rets)
#     return  count_trades,count_profitable_trades,0 if len(profits_bag)==0 else sum(profits_bag)/len(profits_bag),0 if len(losses_bag)==0 else sum(losses_bag)/len(losses_bag),len(profits_bag),len(losses_bag)
    return count_trades,profits_bag,losses_bag,profit_bars,loss_bars

In [ ]:
def get_n_days_ATR(df,n):
    data = df.copy()
    high = data['High']
    low = data['Low']
    close = data['Close']
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = tr.rolling(n).mean()
    df['ATR'] = atr
    df.dropna(inplace = True)

    return 

def get_volatility_sqeeze_signal(df,squeeze_look_back,squeeze_memory):
   
    df['IsVolatilitySqueeze'] = 0
    df.loc[df['BandWidth'] <= df['BandWidth'].rolling(squeeze_look_back).min(),'IsVolatilitySqueeze'] = 1
    df.loc[df['IsVolatilitySqueeze'].rolling(squeeze_memory).sum() > 0,'IsVolatilitySqueeze'] = 1
    df.dropna(inplace = True)
    
    return

def get_BBand_Cross_Signal(df):

    conditions = [
        (df['Close'] <  df['LowerBand']),
        (df['Close'] >  df['UpperBand']),
        ]

    values = [-1, 1]
    df['BandCrossSignal'] = 0
    df['BandCrossSignal'] = np.select(conditions, values)
    df['BandCrossSignal'] = df['BandCrossSignal'].shift(1)
    df.dropna(inplace = True)

    return

def get_bollinger_bands(df,MA_period,band_width):

    df['MA'] = df['Close'].rolling(MA_period).mean()
    df['UpperBand'] = df['MA'] + (df['Close'].rolling(MA_period).std()*band_width)
    df['LowerBand'] = df['MA'] - (df['Close'].rolling(MA_period).std()*band_width)
    df['BandWidth'] = (df['UpperBand'] - df['LowerBand'])/df['MA'] 
    df.dropna(inplace = True)
    
    return   